#1 자연어 처리 학습시키기

In [1]:
from konlpy.tag import Okt
okt = Okt()

In [2]:
# 모델 불러오기(학습데이터: 네이버 영화 평가)
from keras.models import load_model
model = load_model('movie_review_model.h5')

In [3]:
def tokenize(doc):
    #형태소와 품사를 join
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

In [4]:
import pandas as pd
train_df = pd.read_csv("ratings_train.txt", "\t")

In [5]:
train_df.isnull().any() #document에 null값이 있다.
train_df['document'] = train_df['document'].fillna(''); #null값을 ''값으로 대체

In [6]:
#tokenize 과정은 시간이 오래 걸릴수 있음...
train_docs = [(tokenize(row[1]), row[2]) for row in train_df.values]

In [7]:
tokens = [t for d in train_docs for t in d[0]]
print("토큰개수:", len(tokens))

토큰개수: 2159921


In [8]:
import nltk
import numpy as np
text = nltk.Text(tokens, name='NMSC')

In [9]:
FREQUENCY_COUNT = 100; #시간적 여유가 있다면 100개를 해보도록~
selected_words = [f[0] for f in text.vocab().most_common(FREQUENCY_COUNT)]

In [10]:
#영화평가에서 사용된 단어리스트 문서에서 상위 100개들중 포함되는 단어들이 개수
def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

In [11]:
def predict_sentiment(review):
    token = tokenize(review)
    tfq = term_frequency(token)
    data = np.expand_dims(np.asarray(tfq).astype('float32'), axis=0)
    score = float(model.predict(data))
    #if(score > 0.5):
        #print(f"{review} ==> 긍정 ({round(score*100)}%)")
    #else:
        #print(f"{review} ==> 부정 ({round((1-score)*100)}%)")
    return score

In [12]:
predict_sentiment("재미 정말 없어요")

0.07519051432609558

In [13]:
predict_sentiment("괜찮")

0.5579190850257874

In [14]:
predict_sentiment("카카오 주가 전망 오히려 지금이 매력적인 이유")

0.7493253350257874

#2 분석할 댓글 데이터 가져오기

In [15]:
youtube = pd.read_csv('카카오 댓글(20_4~21_7).csv')

In [16]:
youtube

,Stock,Date,Video ID,Comment,Comment Like
0,카카오,2020-07-03,GAj_h1lhoTA,SK증권에서 5/21일 추천 드린 15개 종목은 다음과 같습니다.<br><br>1)...,26.0
1,카카오,2020-07-03,GAj_h1lhoTA,니프티는 글로벌라이제아션때문에 오른거지 내수때매 오른건 아니지안나 진정한 내수주 네...,5.0
2,카카오,2020-07-04,GYUUCCJL5FQ,저는 솔직히 카카오 재섭어서 카드론 받아서 천만원 박았는데 수익나서 이번에 천만원 ...,2.0
3,카카오,2020-07-04,GYUUCCJL5FQ,카카오 담주 월욜에도 오를까요??,1.0
4,카카오,2020-07-04,GYUUCCJL5FQ,오늘 처음 왔어요<br>목소리 짱..설명짱입니다<br>카카오.네이버..지금 사도 괜...,4.0
...,...,...,...,...,...
5797,카카오,2021-07-31,EfHM_K5sTt8,영두형~~ 나 카카오 145000에 100주 재입성했옹 잘한거맞징???,19.0
5798,카카오,2021-07-31,EfHM_K5sTt8,파워풀회장님 임실장님<br>넘 잼나요 중독성 있어욤<br>좋은정보 희망적인 말씀 감...,17.0
5799,카카오,2021-07-31,EKP0vM39oqc,카카오뱅크는 아시다시피 8월 6일 상장입니다.,5.0
5800,카카오,2021-07-31,EKP0vM39oqc,항상 감사드리고 주말에도 수고 많으십니다 많은 분들이 아시겠지만 카카오뱅크공모는 다...,13.0


In [17]:
#1만 조회수 이상인 영상의 댓글들의 감성분석 결과와 점수를 추가
youtube['Sentiment Score'] = 0.0
youtube['Sentiment'] = ""
for i in range(len(youtube)):
    youtube['Sentiment Score'][i] = predict_sentiment(youtube['Comment'][i])
    if youtube['Sentiment Score'][i] > 0.5:
        youtube['Sentiment'][i] = "긍정"
    else:    
        youtube['Sentiment'][i] = "부정"
youtube

<ipython-input-17-46c5d2287979>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youtube['Sentiment Score'][i] = predict_sentiment(youtube['Comment'][i])


,Stock,Date,Video ID,Comment,Comment Like,Sentiment Score,Sentiment
0,카카오,2020-07-03,GAj_h1lhoTA,SK증권에서 5/21일 추천 드린 15개 종목은 다음과 같습니다.<br><br>1)...,26.0,0.482406,긍정
1,카카오,2020-07-03,GAj_h1lhoTA,니프티는 글로벌라이제아션때문에 오른거지 내수때매 오른건 아니지안나 진정한 내수주 네...,5.0,0.255938,긍정
2,카카오,2020-07-04,GYUUCCJL5FQ,저는 솔직히 카카오 재섭어서 카드론 받아서 천만원 박았는데 수익나서 이번에 천만원 ...,2.0,0.362936,긍정
3,카카오,2020-07-04,GYUUCCJL5FQ,카카오 담주 월욜에도 오를까요??,1.0,0.557919,긍정
4,카카오,2020-07-04,GYUUCCJL5FQ,오늘 처음 왔어요<br>목소리 짱..설명짱입니다<br>카카오.네이버..지금 사도 괜...,4.0,0.593762,긍정
...,...,...,...,...,...,...,...
5797,카카오,2021-07-31,EfHM_K5sTt8,영두형~~ 나 카카오 145000에 100주 재입성했옹 잘한거맞징???,19.0,0.499954,긍정
5798,카카오,2021-07-31,EfHM_K5sTt8,파워풀회장님 임실장님<br>넘 잼나요 중독성 있어욤<br>좋은정보 희망적인 말씀 감...,17.0,0.962954,긍정
5799,카카오,2021-07-31,EKP0vM39oqc,카카오뱅크는 아시다시피 8월 6일 상장입니다.,5.0,0.395436,긍정
5800,카카오,2021-07-31,EKP0vM39oqc,항상 감사드리고 주말에도 수고 많으십니다 많은 분들이 아시겠지만 카카오뱅크공모는 다...,13.0,0.322591,긍정


In [22]:
youtube.to_csv("카카오 감성분석.csv",index=False)